# Edge MLOps Monitor Demo

This notebook demonstrates the Edge MLOps Monitor library in a simulated edge device environment using Google Colab.

## Setup

First, let's install our library from PyPI:

In [ ]:
!pip install edge-mlops-monitor scikit-learn pandas numpy matplotlib tqdm

## Create Configuration

Let's create a basic configuration for our monitoring:

In [ ]:
%%writefile config.yaml
system:
  sampling_interval_seconds: 5  # More frequent for demo purposes
  max_memory_buffer_mb: 50

model_logging:
  buffer_size: 1000
  log_level: INFO

drift_detection:
  algorithm: ks_test
  threshold: 0.05
  window_size: 50  # Size of the sliding window for drift detection
  check_frequency: 50  # Check after every 50 predictions
  reference_data: {}  # Will be populated with initial predictions

telemetry:
  upload_interval_seconds: 30  # More frequent for demo
  max_batch_size: 100
  retry_base_delay_seconds: 1
  retry_max_delay_seconds: 60
  retry_max_attempts: 5

storage:
  type: "sqlite"
  sqlite_path: "edge_monitor.db"
  max_sqlite_size_mb: 100

## Create a Simple ML Model

Let's create a simple scikit-learn model to simulate an edge device ML model:

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Create synthetic data
np.random.seed(42)
X = np.random.rand(100, 2)
y = 2 * X[:, 0] + 3 * X[:, 1] + np.random.randn(100) * 0.1

# Train a simple model
model = LinearRegression()
model.fit(X, y)

print("Model trained! Coefficients:", model.coef_)

## Initialize Edge MLOps Monitor

In [ ]:
from edge_mlops_monitor import EdgeMonitor

# Initialize the monitor
monitor = EdgeMonitor(config_path="config.yaml")

# Start system monitoring in the background
monitor.start_system_monitoring()

# Verify monitor is running
print("Monitor initialized successfully!")
print("System monitoring started in background.")
print("SQLite database path:", monitor.config['storage']['sqlite_path'])

## Simulate Edge Device Predictions

Now let's simulate making predictions and monitoring them:

In [ ]:
import time
from tqdm.notebook import tqdm

# Function to simulate predictions
def make_prediction(input_data):
    # Log the input
    input_dict = {
        'feature1': float(input_data[0]),
        'feature2': float(input_data[1])
    }
    
    # Log the input
    input_id = monitor.log_model_input(input_dict)
    
    # Make prediction
    prediction = model.predict([input_data])[0]
    
    # Add some artificial drift over time
    drift_factor = 1.0 + (time.time() % 100) / 1000  # Small increasing drift
    prediction *= drift_factor
    
    # Convert output to dictionary
    output_dict = {
        'prediction': float(prediction)
    }
    
    # Log the output
    monitor.log_model_output(input_id, output_dict)
    
    return prediction

# Simulate predictions over time
n_predictions = 200
print(f"Making {n_predictions} predictions...")

for _ in tqdm(range(n_predictions)):
    # Generate random input
    input_data = np.random.rand(2)
    
    # Make prediction
    prediction = make_prediction(input_data)
    
    # Simulate some processing time
    time.sleep(0.1)  # 100ms between predictions

## Check Monitoring Results

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('edge_monitor.db')

# Get system metrics
df_system = pd.read_sql_query("SELECT * FROM system_metrics", conn)
print("\nSystem Metrics Summary:")
print(df_system.describe())

# Get model inputs/outputs
df_model = pd.read_sql_query("SELECT * FROM model_io", conn)
print("\nModel I/O Summary:")
print(f"Total predictions: {len(df_model)}")

# Get drift detection results
df_drift = pd.read_sql_query("SELECT * FROM drift_detection", conn)
print("\nDrift Detection Results:")
if len(df_drift) > 0:
    print(df_drift)
else:
    print("No drift detected yet")

conn.close()

## Visualize Monitoring Data

In [ ]:
import matplotlib.pyplot as plt

# Plot system metrics over time
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

ax1.plot(df_system['timestamp'], df_system['cpu_percent'], label='CPU Usage')
ax1.set_title('CPU Usage Over Time')
ax1.set_xlabel('Timestamp')
ax1.set_ylabel('CPU %')
ax1.legend()

ax2.plot(df_system['timestamp'], df_system['memory_percent'], label='Memory Usage')
ax2.set_title('Memory Usage Over Time')
ax2.set_xlabel('Timestamp')
ax2.set_ylabel('Memory %')
ax2.legend()

plt.tight_layout()
plt.show()

## Cleanup

Stop the monitor and clean up:

In [ ]:
# Stop the monitor
monitor.stop()

print("Monitoring stopped and cleaned up!")